In [16]:
import torch
import random
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

In [2]:
from transformers import logging
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from tokenizers import ByteLevelBPETokenizer
logging.set_verbosity_warning()

In [4]:
device = 'cpu'
if (torch.backends.mps.is_available()):
    print('MPS: ' + str(torch.backends.mps.is_available()))
    device = 'mps'
if (torch.cuda.is_available()):
    print('CUDA: ' + str(torch.cuda.is_available()))
    device = 'cuda'
device = 'cpu'
print('Using device: '+device)


MPS: True
Using device: cpu


In [5]:
# spt_model  = AutoModelForCausalLM.from_pretrained("datificate/gpt2-small-spanish")

In [6]:
# model_diputados = torch.load('./model/gpt2-small-diputados/pytorch_model.bin')
model_diputados = GPT2LMHeadModel.from_pretrained('./model/gpt2-small-diputados')
# tokenizer_diputados = ByteLevelBPETokenizer.from_pretrained('./model/gpt2-small-diputados')

tokenizer_diputados = GPT2TokenizerFast.from_pretrained(
    str('./model/ByteLevelBPE_tokenizer_es'), 
    pad_token='<|endoftext|>')
tokenizer_diputados.model_max_length = 1024

# model_diputados.lm_head.weight = spt_model.transformer.wte.weight

In [7]:
print(torch.backends.mps.is_available())


True


In [8]:
model_diputados.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [9]:
top_k = 50

model_diputados.eval();
model_diputados.to(device);

In [12]:
text = 'En las últimas semanas hemos visitado varias zonas de Cartago y hemos visto el lamentable, por no decir vergonzoso, estado de las carreteras. Hablo de gente que debe lidiar con presas eternas, agricultores que se juegan la vida para sacar sus productos, de niños que no pueden llegar a tiempo a sus escuelas. Hablo de personas que se sienten olvidadas por sus gobiernos desde hace muchos años atrás.'
input_ids = tokenizer_diputados.encode(text, return_tensors="pt").to(device)
print(input_ids.shape)
max_length = input_ids.shape[1]
flat_input_ids = torch.flatten(input_ids,start_dim = 1)
print(flat_input_ids.shape)

print(max_length)

torch.Size([1, 75])
torch.Size([1, 75])
75


In [13]:
textos = model_diputados.generate(input_ids, pad_token_id=50256,
                                   do_sample=True, 
                                   max_length=400, 
                                   min_length=400,
                                   top_k=50,
                                   num_return_sequences=10)

In [17]:
for i, sample_output in enumerate(textos):
    salida_texto = tokenizer_diputados.decode(sample_output.tolist())
    print(">> Generated text {}\n\n{}".format(i+1, salida_texto))
    seq = random.randint(0,100000)
#     with open('/content/textos/ejemplo_diputado_'+str(seq)+'.txt','w') as f:
#       f.write(salida_texto)
    print('\n---')

>> Generated text 1

En las últimas semanas hemos visitado varias zonas de Cartago y hemos visto el lamentable, por no decir vergonzoso, estado de las carreteras. Hablo de gente que debe lidiar con presas eternas, agricultores que se juegan la vida para sacar sus productos, de niños que no pueden llegar a tiempo a sus escuelas. Hablo de personas que se sienten olvidadas por sus gobiernos desde hace muchos años atrás.
Yo les pido de verdad que apoyemos esta iniciativa y que lo demos, me llena de regocijo. Realmente el corazón tiene que ver con el corazón.Acta de la sesión plenaria ordinaria N
.º
 49
Lunes 13 de agosto de 2018
Primer período de sesiones ordinarias
Primera legislaturaDiputado Pablo Heriberto Abarca Mora:
Gracias, presidente.
El diputado Abarca presentó un proyecto de ley de forma tal que va a ser un gran logro de algunas fracciones, aquí se quiere poner la palabra para tratar de mejorarlo, y para tratar de mejorarlo.
Se quiere también poner una moción de censura que es la

# Tratando con Learner

In [ ]:
def splitter(model):
    
    modules = []
    for i in range(4): modules.append(model.transformer.h[i])
    groups = [nn.Sequential(*modules)]

    modules = []
    for i in range(4,8,1): modules.append(model.transformer.h[i])
    groups = L(groups + [nn.Sequential(*modules)])

    modules = []
    for i in range(8,12,1): modules.append(model.transformer.h[i])
    groups = L(groups + [nn.Sequential(*modules)])
    

    groups = L(groups +    [nn.Sequential(model.transformer.wte,model.transformer.wpe,model.transformer.ln_f)])
    
    return groups.map(params)

class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))
        
df = pd.read_csv('./model/diputados.csv')
df.dropna()
print(len(df))
dfl = df[0:2000]['text'].tolist()
print(len(dfl))
        
num = int(0.8*len(df)) # 80% para entrenamiento
idxs = np.random.randint(0, len(dfl), len(dfl))
idxs_train = idxs[:num]
idxs_val = idxs[num:]
# We gather all texts in one numpy array 
# (since it will be easier to use this way with fastai)
# all_texts = np.concatenate([dfl.iloc[idxs_train], dfl.iloc[idxs_val]])
all_texts = np.concatenate([dfl[:num],dfl[num:]])
# all_texts = np.concatenate([df.iloc[idxs_train].text.astype(str).values.to_list(), 
#                             df.iloc[idxs_val].text.astype(str).values.to_list()])
splits = [list(idxs_train), list(idxs_val)]
tls = TfmdLists(all_texts, TransformersTokenizer(tokenizer_diputados), splits=splits, dl_type=LMDataLoader)
        
bs,sl = 4,1024
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [ ]:
# Tratando con Learner...
from fastai.text.all import *

learn = Learner(dls, model_diputados, loss_func=CrossEntropyLossFlat(),
                splitter = splitter,
                cbs=[DropOutput], 
                metrics=[accuracy, Perplexity()]).to_fp16()
learn = learn.load('./modelo_diputados/GPT2_dp_5epoch')

gpt2_small_diputados = learn.model

In [ ]:
text = 'En las últimas semanas hemos visitado varias zonas de Cartago y hemos visto el lamentable, por no decir vergonzoso, estado de las carreteras. Hablo de gente que debe lidiar con presas eternas, agricultores que se juegan la vida para sacar sus productos, de niños que no pueden llegar a tiempo a sus escuelas. Hablo de personas que se sienten olvidadas por sus gobiernos desde hace muchos años atrás.'
input_ids = tokenizer_diputados.encode(text, return_tensors="pt").to('mps')
print(input_ids.shape)
max_length = input_ids.shape[1]
flat_input_ids = torch.flatten(input_ids,start_dim = 1)
print(flat_input_ids.shape)

print(max_length)

In [ ]:

textos = gpt2_small_diputados.generate(flat_input_ids, pad_token_id=50256,
                                   do_sample=True, 
                                   max_length=400, 
                                   min_length=400,
                                   top_k=50,
                                   num_return_sequences=10)